<a href="https://colab.research.google.com/github/kareemullah1234/AI_Agent_content/blob/main/Agent_7_Paper_MCP_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =============================
# 1. Install Dependencies
# =============================
!pip install groq arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.7 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=ef43f9ab9fec88b15ad556bfe2e7a6560171f61cad23342561bc0fa5a931dc7d
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k


In [2]:


# =============================
# 2. Imports & Config
# =============================
import os
import arxiv
import json
from typing import List
from groq import Groq

# --- Hardcoded API key ---
os.environ["GROQ_API_KEY"] = ""

# Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])
MODEL = "llama3-70b-8192"   # You can also try "mixtral-8x7b-32768"

# Directory to store papers
PAPER_DIR = "papers"
os.makedirs(PAPER_DIR, exist_ok=True)

# =============================
# 3. Paper Search Function
# =============================
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    client_arxiv = arxiv.Client()
    search = arxiv.Search(
        query=topic,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )

    papers = client_arxiv.results(search)
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)

    file_path = os.path.join(path, "papers_info.json")
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            "title": paper.title,
            "authors": [author.name for author in paper.authors],
            "summary": paper.summary,
            "pdf_url": paper.pdf_url,
            "published": str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info

    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)

    print(f"Results are saved in: {file_path}")
    return paper_ids

# =============================
# 4. Extract Paper Info
# =============================
def extract_info(paper_id: str) -> str:
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError):
                    continue
    return f"There’s no saved information related to paper {paper_id}."

# =============================
# 5. Process Query with Groq
# =============================
def process_query(query: str):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": query}],
        temperature=0.7,
        max_tokens=1024,
    )
    reply = response.choices[0].message.content
    print(reply)
    return reply




In [3]:
# =============================
# 6. Example Run
# =============================
# Search for papers
ids = search_papers("machine learning", max_results=3)
print("Paper IDs:", ids)

# Extract info
print(extract_info(ids[0]))

# Ask Groq model
process_query("Summarize the key contributions of recent papers on machine learning")

Results are saved in: papers/machine_learning/papers_info.json
Paper IDs: ['1909.03550v1', '1811.04422v1', '1707.04849v1']
{
  "title": "Lecture Notes: Optimization for Machine Learning",
  "authors": [
    "Elad Hazan"
  ],
  "summary": "Lecture notes on optimization for machine learning, derived from a course at\nPrinceton University and tutorials given in MLSS, Buenos Aires, as well as\nSimons Foundation, Berkeley.",
  "pdf_url": "http://arxiv.org/pdf/1909.03550v1",
  "published": "2019-09-08"
}
What a monumental task! There have been countless papers on machine learning in recent years, and it's challenging to summarize the key contributions of all of them. However, I'll try to provide an overview of some influential papers in various areas of machine learning, highlighting their key contributions:

**Deep Learning**

1. **ResNet** (He et al., 2016): Introduced the concept of residual connections, which enable training of deeper neural networks and achieve state-of-the-art performa

"What a monumental task! There have been countless papers on machine learning in recent years, and it's challenging to summarize the key contributions of all of them. However, I'll try to provide an overview of some influential papers in various areas of machine learning, highlighting their key contributions:\n\n**Deep Learning**\n\n1. **ResNet** (He et al., 2016): Introduced the concept of residual connections, which enable training of deeper neural networks and achieve state-of-the-art performance on image classification tasks.\n2. **BERT** (Devlin et al., 2019): Developed a language model that uses a multi-layer bidirectional transformer to achieve state-of-the-art results on a wide range of natural language processing tasks, including question answering, sentiment analysis, and text classification.\n3. **Transformers** (Vaswani et al., 2017): Introduced the transformer architecture, which has become a standard component in many state-of-the-art models for natural language processin